<a href="https://colab.research.google.com/github/ajiglover/DS-Projects/blob/main/Copy_of_Duplicate_Clustering_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Duplicate Prediction Model


Deduplication Optimization ✨

Today, I'm working on an open-source duplicate prediction model designed to help organizations identify and resolve duplicate records in their entity data. The number of comparisons grows as n(n-1)/2, where n represents the total number of records. My best result so far achieves 34 million comparisons per second on a single commodity GPU. This performance is attained using a TF/IDF matrix with a cosine similarity metric, applied through a nearest-neighbors algorithm. Processing 100,000 records took 144 seconds to fit, which is acceptable. However, scaling up to 1,000,000 records would require over 4 hours and involve half a trillion comparisons.

ChatGPT suggests that FAISS from Facebook and Annoy from Spotify might be competitive alternatives. However, given the nature of the n(n-1)/2 comparison growth, I'm not convinced the improvement would be significant. I believe a more promising direction would be to scale the task with Spark. In the meantime, I'm planning to stick with the current algorithm for now and focus on building out some masking for larger datasets.

For generating test data with labeled duplicates, I've been using Mimesis, which is quite efficient in creating mock data.

Is anyone else working on something similar? I'd love to exchange insights and discuss the best approaches.  Here's a copy of my notebook:


In [ ]:
!pip install mimesis # used for generating test data

## Sample Data

In [ ]:
import random
import pandas as pd

### First Name Variations

In [ ]:
first_name_variations = {
    'Dave': 'David',
    'Joe': 'Joseph',
    'Elizabeth': 'Betty',
    'Bill': 'William',
    'Bob': 'Robert',
    'Tom': 'Thomas',
    'Dick': 'Richard',
    'Jim': 'James',
    'Jenny': 'Jennifer',
    'Kathy': 'Katherine',
    'Sue': 'Susan',
    'Maggie': 'Margaret',
    'Harry': 'Harold',
    'Ron': 'Ronald',
    'Nick': 'Nicholas',
    'Patty': 'Patricia',
    'Chris': 'Christopher',
    'Mike': 'Michael',
    'Steve': 'Steven',
    'Andy': 'Andrew',
    'Matty': 'Matthew',
    'Sam': 'Samuel',
    'Tony': 'Anthony',
}

In [ ]:
def first_name_variation(full_name, chance=40):
    name_parts = full_name.split()
    first_name = name_parts[0]
    if random.random() < (chance / 100.0):
        if first_name in first_name_variations:
            first_name = first_name_variations[first_name]
        else:
            for key, value in first_name_variations.items():
                if first_name == value:
                    first_name = key
                    break
    return ' '.join([first_name] + name_parts[1:])

print(first_name_variation("Joe Satriani",chance=100))
print(first_name_variation("David Bowie",chance=100))

### Last Name Transpositions

In [ ]:
def last_name_transposition(full_name, chance=1):
    name_parts = full_name.split()
    if len(name_parts) > 1 and random.random() < (chance / 100.0):
        last_name = name_parts[-1]
        if len(last_name) > 4:
            i = random.randint(1, len(last_name) - 2)
            last_name_chars = list(last_name)
            last_name_chars[i], last_name_chars[i+1] = \
              last_name_chars[i+1], last_name_chars[i]
            name_parts[-1] = ''.join(last_name_chars)
    return ' '.join(name_parts)
print(last_name_transposition("Jon Anderson", chance=100))

### Generate N Rows

In [ ]:
from mimesis import Person, Address, Finance

def mimesis_generate_data(n):
    data = []
    entity_id = 0

    person = Person('en')
    address_gen = Address('en')
    finance = Finance('en')

    for _ in range(n):
        entity_id += 1
        is_person = True
        name = person.full_name()
        if random.random() < (10.0 / 100.0):
            is_person = False
            name = finance.company()
        street_address = address_gen.address()
        city = address_gen.city()
        state = address_gen.state(abbr=True)
        label = 0  # not a duplicate
        data.append([entity_id, name, street_address, city, state, label])
        if is_person:
            dup_name = first_name_variation(name, chance=30)
            dup_name = last_name_transposition(dup_name, chance=1)
            if name != dup_name:
                entity_id += 1
                label = 1
                data.append([entity_id, dup_name, street_address, city, state, label])

    return data

mimesis_mock_data = mimesis_generate_data(100000)
mimesis_df = pd.DataFrame(mimesis_mock_data,
    columns=['entity_id', 'name', 'address', 'city', 'state', 'label'])
#mimesis_df

## TF/IDF & Nearest Neighbors Cosine


In [ ]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import NearestNeighbors

df = mimesis_df.sort_values('entity_id', ascending=False).reset_index(drop=True)
df['combined'] = df[['name', 'address', 'city', 'state']].agg(' '.join, axis=1)
df['dup_id'] = df['entity_id']
df['dup_flag'] = 0
df['score'] = 0.0

vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(df['combined'])

# Use NearestNeighbors to find similarities above the threshold
threshold = 0.7
nbrs = NearestNeighbors(radius=threshold, metric='cosine').fit(tfidf_matrix)
distances, indices = nbrs.radius_neighbors(tfidf_matrix)

for i, (dist, idx) in enumerate(zip(distances, indices)):
    for d, j in zip(dist, idx):
        if i < j:  # only compare with what comes after:
            similarity_score = 1 - d  # Convert distance to similarity
            if similarity_score > threshold:
                df.loc[i, 'dup_flag'] = 1
                df.loc[j, 'dup_flag'] = 2
                if df.loc[j, 'dup_id'] == df.loc[j, 'entity_id']:
                    df.loc[j, 'dup_id'] = df.loc[i, 'entity_id']
                    df.loc[j, 'score'] = similarity_score


In [ ]:
crit = (df['dup_flag'] > 0) | (df['label'] > 0)
df[crit][['entity_id','dup_id','dup_flag','score','label',
          'name','address','city','state','combined']
         ].sort_values(['dup_id','dup_flag'], ascending=[False, True])